In [ ]:
import os
import yaml

import numpy as np
from stl import mesh

# from conteva import Conteva

# bounding_box_filename = '/home/tmn/ws_caric/src/caric_mission/models/mbs/bounding_boxes/box_description.yaml'
# bounding_box_filename = '/home/tmn/ws_caric/src/caric_mission/models/hangar/bounding_boxes/box_description.yaml'
bounding_box_filename = '/home/tmn/ws_caric/src/caric_mission/models/crane/bounding_boxes/box_description.yaml'

def generate_stl_box(center, size, orientation, filename):
    
    # Calculate the half sizes
    half_sizes = size / 2.0
    
    # Define the eight vertices of the rectangle
    vertices = np.array([ [-half_sizes[0], -half_sizes[1], -half_sizes[2]],
                          [ half_sizes[0], -half_sizes[1], -half_sizes[2]],
                          [ half_sizes[0],  half_sizes[1], -half_sizes[2]],
                          [-half_sizes[0],  half_sizes[1], -half_sizes[2]],
                          [-half_sizes[0], -half_sizes[1],  half_sizes[2]],
                          [ half_sizes[0], -half_sizes[1],  half_sizes[2]],
                          [ half_sizes[0],  half_sizes[1],  half_sizes[2]],
                          [-half_sizes[0],  half_sizes[1],  half_sizes[2]]
                        ])

    # Rotate the vertices around the center
    rotation_matrix = orientation
    rotated_vertices = np.dot(vertices, rotation_matrix.T)

    # Translate the rotated vertices to the center position
    translated_vertices = rotated_vertices + center

    # Create the mesh, each triang has two sides
    faces = np.array([
                        # Bottom face
                        [0, 1, 2],
                        [0, 2, 3],
                        [0, 2, 1],
                        [0, 3, 2],
                        # Top face
                        [4, 5, 6],
                        [4, 6, 7],
                        [4, 6, 5],
                        [4, 7, 6],
                        # Side faces
                        [0, 1, 5],
                        [0, 4, 5],
                        [0, 5, 1],
                        [0, 5, 4],
                        [1, 2, 6],
                        [1, 5, 6],
                        [1, 6, 2],
                        [1, 6, 5],
                        [2, 3, 7],
                        [2, 6, 7],
                        [2, 7, 3],
                        [2, 7, 6],
                        [3, 0, 4],
                        [3, 7, 4],
                        [3, 4, 0],
                        [3, 4, 7]
                     ])

    # Create the STL mesh object
    mesh_data = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, face in enumerate(faces):
        for j, vertex_index in enumerate(face):
            mesh_data.vectors[i][j] = translated_vertices[vertex_index]

    # Save the mesh as an STL file
    mesh_data.save(filename)

In [ ]:
f = open(bounding_box_filename, "r")
boxes = yaml.safe_load(f)

for box in boxes:
    center      = boxes[box]['center']
    size        = boxes[box]['size']
    orientation = np.array(boxes[box]['orientation']).reshape(4, 4)
    
    print("box name: ",   center     )
    print("box size: ",   size       )
    print("box orie: \n", orientation)

    generate_stl_box(np.array(center), np.array(size), orientation[0:3, 0:3],
                     os.path.dirname(bounding_box_filename) + '/' + box + '.stl')
